In [ ]:
from embedding_search.vector_store import MiniStore
import logging

logging.basicConfig(level=logging.INFO)
store = MiniStore()
store.build()

# Feature 1: Search related articles

In [ ]:
articles = store.search("pandemic resilience", type="article")
[print(article.title) for article in articles]

In [ ]:
# This search is based on the author's articles centroid, i.e., the average of all the articles' embeddings
# i.e., it is based on relevancy only, not weighted by the number of articles
authors = store.search("higgs boson", type="author")
[print(author) for author in authors]

In [ ]:
# This is a weighted by articles related to the query
authors = store.weighted_search_author("Higgs field", n_pool=100)
[print(author) for author in authors]

# Network graph
The purpose of this network graph prototype is to visualize a author and their works.

In [ ]:
from embedding_search.visualize import EmbeddingsProcessor, QueryPlotter

In [ ]:
STORE = store  # cache this...


def plot(query: str):
    processor = EmbeddingsProcessor(STORE)
    plotter = QueryPlotter(processor)
    return plotter.plot(query)

In [ ]:
query = "higgs boson"
p = plot(query)
p.save(f"plots/{query}.html")